In [15]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import random as rd
import warnings
warnings.filterwarnings("ignore")
from geopy.geocoders import Nominatim

In [6]:
geolocator = Nominatim(user_agent="geospatial-analysis-kaggle")

In [7]:
location = geolocator.geocode("Pyramid of Khufu")
# print([location.point.latitude, location.point.longitude])
# print(location.address)

In [9]:
universities = pd.read_csv("../../geospatial-learn-course-data/top_universities.csv")
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None

universities[['Latitude', 'Longitude']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(universities["Latitude"])) / len(universities)) * 100))

# Drop universities that were not successfully geocoded
universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(
    universities, geometry=gpd.points_from_xy(universities.Longitude, universities.Latitude))
universities.crs = {'init': 'epsg:4326'}
universities.head()

88.0% of addresses were geocoded!


,Name,Latitude,Longitude,geometry
0,University of Oxford,51.758708,-1.255668,POINT (-1.25567 51.75871)
3,ETH Zurich,47.413218,8.537491,POINT (8.53749 47.41322)
4,UCL,50.851809,4.453697,POINT (4.4537 50.85181)
5,London School of Economics and Political Science,51.514598,-0.116422,POINT (-0.11642 51.5146)
6,University of Edinburgh,55.944076,-3.188374,POINT (-3.18837 55.94408)


In [52]:
def showMap(data):
    map = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)
    for _, university in data.iterrows():
        color = f"rgb({rd.randint(0, 255)}, {rd.randint(0, 255)}, {rd.randint(0, 255)})"
        # Choix d'une icône FontAwesome aléatoire parmi une liste prédéfinie
        icons = ['fa-university', 'fa-school', 'fa-graduation-cap']
        icon = rd.choice(icons)
        folium.Marker(location=[university["Latitude"], university["Longitude"]], icon=folium.Icon(color=color, icon=icon, prefix='fa')).add_to(map)
    return map
showMap(universities)

In [46]:
world = gpd.read_file("https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip")
europe = world.loc[world.CONTINENT == 'Europe'].reset_index(drop=True)
europe_stats = europe[["NAME", "POP_EST", "GDP_MD"]]
europe_boundaries = europe[["NAME", "geometry"]]

display(europe_boundaries)

,NAME,geometry
0,Russia,"MULTIPOLYGON (((178.7253 71.0988, 180 71.51571..."
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80..."
2,France,"MULTIPOLYGON (((-51.6578 4.15623, -52.24934 3...."
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239..."
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834..."
5,Ukraine,"POLYGON ((31.78599 52.10168, 32.15944 52.06125..."
6,Poland,"POLYGON ((23.48413 53.9125, 23.52754 53.47012,..."
7,Austria,"POLYGON ((16.97967 48.1235, 16.90375 47.71487,..."
8,Hungary,"POLYGON ((22.08561 48.42226, 22.64082 48.15024..."
9,Moldova,"POLYGON ((26.61934 48.22073, 26.85782 48.36821..."


In [48]:
europe_boundaries_stats = europe_boundaries.merge(europe_stats, on="NAME")
display(europe_boundaries_stats)

,NAME,geometry,POP_EST,GDP_MD
0,Russia,"MULTIPOLYGON (((178.7253 71.0988, 180 71.51571...",144373535.0,1699876
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80...",5347896.0,403336
2,France,"MULTIPOLYGON (((-51.6578 4.15623, -52.24934 3....",67059887.0,2715518
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239...",10285453.0,530883
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834...",9466856.0,63080
5,Ukraine,"POLYGON ((31.78599 52.10168, 32.15944 52.06125...",44385155.0,153781
6,Poland,"POLYGON ((23.48413 53.9125, 23.52754 53.47012,...",37970874.0,595858
7,Austria,"POLYGON ((16.97967 48.1235, 16.90375 47.71487,...",8877067.0,445075
8,Hungary,"POLYGON ((22.08561 48.42226, 22.64082 48.15024...",9769949.0,163469
9,Moldova,"POLYGON ((26.61934 48.22073, 26.85782 48.36821...",2657637.0,11968


In [56]:
europe_universities = gpd.sjoin(universities, europe)
european_universities = gpd.sjoin(universities, europe)

# Investigate the result
print(f"We located {len(universities)} universities.")
print(f"Only {len(european_universities)} of the universities were located in Europe (in {len(european_universities.NAME.unique())} different countries).")

showMap(europe_universities)

We located 88 universities.
Only 84 of the universities were located in Europe (in 15 different countries).
